In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [14]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
#lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
print(lookup_tensor)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([0])
tensor([[ 1.8782, -0.5666,  0.4016, -0.1153,  0.3170]],
       grad_fn=<EmbeddingBackward>)


## N-Gram

In [ ]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [16]:
test_sentence

['When',
 'forty',
 'winters',
 'shall',
 'besiege',
 'thy',
 'brow,',
 'And',
 'dig',
 'deep',
 'trenches',
 'in',
 'thy',
 "beauty's",
 'field,',
 'Thy',
 "youth's",
 'proud',
 'livery',
 'so',
 'gazed',
 'on',
 'now,',
 'Will',
 'be',
 'a',
 "totter'd",
 'weed',
 'of',
 'small',
 'worth',
 'held:',
 'Then',
 'being',
 'asked,',
 'where',
 'all',
 'thy',
 'beauty',
 'lies,',
 'Where',
 'all',
 'the',
 'treasure',
 'of',
 'thy',
 'lusty',
 'days;',
 'To',
 'say,',
 'within',
 'thine',
 'own',
 'deep',
 'sunken',
 'eyes,',
 'Were',
 'an',
 'all-eating',
 'shame,',
 'and',
 'thriftless',
 'praise.',
 'How',
 'much',
 'more',
 'praise',
 "deserv'd",
 'thy',
 "beauty's",
 'use,',
 'If',
 'thou',
 'couldst',
 'answer',
 "'This",
 'fair',
 'child',
 'of',
 'mine',
 'Shall',
 'sum',
 'my',
 'count,',
 'and',
 'make',
 'my',
 'old',
 "excuse,'",
 'Proving',
 'his',
 'beauty',
 'by',
 'succession',
 'thine!',
 'This',
 'were',
 'to',
 'be',
 'new',
 'made',
 'when',
 'thou',
 'art',
 'old,',
 

In [15]:
# 입력 데이터 토큰화, 
# ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [21]:
vocab = set(test_sentence)
print(vocab)

{"deserv'd", 'shame,', 'now,', 'trenches', 'it', 'praise.', 'where', 'in', 'shall', "excuse,'", 'to', 'own', 'weed', "'This", 'see', 'so', 'thou', 'much', 'Shall', 'count,', 'winters', 'my', 'all', 'How', "youth's", 'answer', 'gazed', 'livery', 'Were', 'thine', 'Will', "beauty's", 'held:', 'warm', 'This', 'proud', 'couldst', 'eyes,', 'Then', 'be', 'Proving', 'his', 'old', 'lusty', 'days;', 'thriftless', 'a', 'Where', 'dig', "totter'd", 'When', 'made', 'besiege', 'sunken', 'when', 'brow,', 'more', 'field,', 'make', 'of', 'Thy', 'praise', 'beauty', 'treasure', 'fair', 'and', 'old,', 'blood', 'lies,', 'new', 'child', 'succession', 'sum', 'small', 'asked,', 'within', "feel'st", 'by', 'To', 'forty', 'cold.', 'mine', 'thine!', 'deep', 'an', 'And', 'worth', 'the', 'being', 'say,', 'thy', 'use,', 'art', 'were', 'If', 'all-eating', 'on'}


In [25]:
word_to_ix = {word : i for i, word in enumerate(vocab)}

In [27]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [28]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. model을 통과할 입력 데이터 준비 
        # (단어를 정수로 변환하고 그것들을 tensor에 감싼다)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. 새로운 instance를 통과시키기 전에 축적된 gradients를 터치에 Recall 
        # 이전 instance로 gradients를 Zero out
        model.zero_grad()

        # Step 3. Model에 forward
        log_probs = model(context_idxs)

        # Step 4. loss function 적용. (loss_function(y_pred, y))
        # y도 tensor에 정수화해서 넣어줌
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. backward에 넣고 gredient 업데이트
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!